# Figure 1-I

In [ ]:
from libraries import *
from parameters import *


In [ ]:
os.getcwd()
os.chdir(projectDir)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
adata = sc.read(par_save_filename_8)

In [ ]:
# results = sc.get.rank_genes_groups_df(adata, group = None)
# results = results.loc[results.pvals_adj < 0.05,]
# results = results.loc[results.logfoldchanges > 0.6,]
# k = results.groupby('group').head(100)
# k.to_csv("/home/beraslan/jovian-work/analysisSingle/Leiden_top100Markers.csv", index=False)

In [ ]:
adata.obs["subCellType"] = "DC2"
adata.obs.loc[adata.obs.leiden.isin(['3']), "subCellType"] = "MacDC"
adata.obs.loc[adata.obs.leiden.isin(['8']), "subCellType"] = "DC1"
adata.obs.loc[adata.obs.leiden.isin(['5']), "subCellType"] = "MReg"

In [ ]:
covariates = adata1.uns['feature_barcode_names_filtered_GENES']
covariates = covariates[covariates != "GENE_CONTROL_"]
covariates = covariates[covariates != "GENE_INEFFECT_"]

In [ ]:
fBarMat = adata.obs[covariates]

In [ ]:
fBarMat[fBarMat>0] = 1
fBarMat["subCellType"] = adata.obs['subCellType']
fBarMat.shape

In [ ]:
allGuidesPerSCT = pd.DataFrame()

for elem in covariates:
    k = pd.DataFrame(pd.crosstab(fBarMat[elem], fBarMat.subCellType))
    k = k.loc[k.index == 1,]
    k["KOGuide"] = elem
    allGuidesPerSCT = allGuidesPerSCT.append(k)
    

In [ ]:
NoOfCellsPerSubCelltype = allGuidesPerSCT[["DC1", "DC2", "MReg", "MacDC"]].sum(axis=0)
NoOfCellsPerSubCelltype

In [ ]:
allGuidesPerSCT['noOfGuideCells'] = allGuidesPerSCT.loc[:,["DC1", "DC2", "MReg", "MacDC"]].sum(axis=1)

In [ ]:
%%R  -i allGuidesPerSCT

lappend <- function(lst, obj) {
  lst[[length(lst)+1]] <- obj
  return(lst)
}

allGuidesPerSCT$KOGuide <- as.character(allGuidesPerSCT$KOGuide)
rownames(allGuidesPerSCT) <- allGuidesPerSCT$KOGuide

for(i in c("DC1", "DC2", "MReg", "MacDC")){
    allGuidesPerSCT[paste0(i,"_pval_greater")] = -1
    allGuidesPerSCT[paste0(i,"_estimate_greater")] = -1
    allGuidesPerSCT[paste0(i,"_pval_less")] = -1
    allGuidesPerSCT[paste0(i,"_estimate_less")] = -1
    
}


totalCountsPerSCT <- data.frame(subCelltype = c("DC1", "DC2", "MReg", "MacDC"),
                                cellCounts = colSums(allGuidesPerSCT[,c("DC1", "DC2", "MReg", "MacDC")]))
rownames(totalCountsPerSCT) = totalCountsPerSCT$subCelltype
totalNoOfCells = sum(totalCountsPerSCT$cellCounts)


depletedGuides <- list()

for (elem in as.character(unique(allGuidesPerSCT$KOGuide))){
    
    
    if(allGuidesPerSCT[elem, "noOfGuideCells"] == 0){
        depletedGuides <- lappend(depletedGuides, elem)
    }else{
        
        for(subCellType in c("DC1", "DC2", "MReg", "MacDC")){
           myDF = data.frame(clusterNo = c(allGuidesPerSCT[elem, subCellType], 
                                           allGuidesPerSCT[elem, "noOfGuideCells"] - allGuidesPerSCT[elem, subCellType]),
                  restNo = c(totalCountsPerSCT[subCellType,"cellCounts"], 
                             totalNoOfCells - totalCountsPerSCT[subCellType,"cellCounts"]))
        
           #print(myDF)
           testRes_g <- fisher.test(x= myDF, alternative = "greater")
           #print(testRes_g)
           allGuidesPerSCT[elem, paste0(subCellType,"_pval_greater")] = testRes_g$p.value
           allGuidesPerSCT[elem, paste0(subCellType,"_estimate_greater")] = testRes_g$estimate
                        
           testRes_l <- fisher.test(x= myDF, alternative = "less")
           #print(testRes_l)
           allGuidesPerSCT[elem, paste0(subCellType,"_pval_less")] = testRes_l$p.value
           allGuidesPerSCT[elem, paste0(subCellType,"_estimate_less")] = testRes_l$estimate
                        
        }
        
    }  
}

for(i in c("DC1", "DC2", "MReg", "MacDC")){
    allGuidesPerSCT[,paste0(i,"_FDR_greater")] <- p.adjust(allGuidesPerSCT[,paste0(i,"_pval_greater")])
    allGuidesPerSCT[,paste0(i,"_FDR_less")] <- p.adjust(allGuidesPerSCT[,paste0(i,"_pval_less")])  
}


#saveRDS(allGuidesPerSCT, "/home/beraslan/jovian-work/analysisSingle/outputs/RDSFiles/allGuidesPerSCT_withTests_PERGENE.rds")


In [ ]:
%%R -w 2.5 -h 10 -u in

library("reshape2")

allGuidesPerSCT_FDRs <- allGuidesPerSCT[,c("DC1_FDR_greater", "DC2_FDR_greater","MReg_FDR_greater", "MacDC_FDR_greater",
                                          "DC1_FDR_less", "DC2_FDR_less", "MReg_FDR_less", "MacDC_FDR_less", "KOGuide")]

allGuidesPerSCT_FDRs_melted <- melt(allGuidesPerSCT_FDRs, id.vars="KOGuide")
allGuidesPerSCT_FDRs_melted <- allGuidesPerSCT_FDRs_melted[allGuidesPerSCT_FDRs_melted$value < 0.15,]

allGuidesPerSCT_FDRs_melted$celltype <- sapply(allGuidesPerSCT_FDRs_melted$variable,
                                               function(x){strsplit(as.character(x),"_")[[1]][1]})
allGuidesPerSCT_FDRs_melted$testType <- sapply(allGuidesPerSCT_FDRs_melted$variable,
                                               function(x){strsplit(as.character(x),"_")[[1]][3]})

allGuidesPerSCT_FDRs_melted$oddRatio = 1

for(i in 1:nrow(allGuidesPerSCT_FDRs_melted)){
    allGuidesPerSCT_FDRs_melted[i,"oddRatio"] = allGuidesPerSCT[allGuidesPerSCT_FDRs_melted[i,"KOGuide"], 
                                                                paste0(allGuidesPerSCT_FDRs_melted[i,"celltype"], "_estimate_", allGuidesPerSCT_FDRs_melted[i,"testType"])]
}   


kk <- data.frame(matrix(1.0, nrow= 4, ncol=length(unique(allGuidesPerSCT_FDRs_melted$KOGuide))))
colnames(kk) = unique(allGuidesPerSCT_FDRs_melted$KOGuide)
rownames(kk) = c("DC1", "DC2", "MReg", "MacDC")


for(i in 1:nrow(allGuidesPerSCT_FDRs_melted)){
        kk[allGuidesPerSCT_FDRs_melted[i, "celltype"],  
           allGuidesPerSCT_FDRs_melted[i, "KOGuide"]] = allGuidesPerSCT_FDRs_melted[i, "oddRatio"]
}

dim(kk)

kk[kk > 2] = 2
# kk = t(kk)

kk <- kk[c("DC2", "MacDC", "MReg", "DC1"),]
kk <- data.frame(t(kk))
kk = kk[order(-kk$DC2, kk$MacDC, kk$MReg, -kk$DC1),]

rownames(kk) <- sapply(rownames(kk), function(x){strsplit(x, "_")[[1]][2]})
library(pheatmap)






In [ ]:
%%R -w 10 -h 2.5 -u in

library(pheatmap)
#pdf(width=6, height=18)
pheatmap(t(kk),cluster_rows=FALSE, cluster_cols=FALSE, treeheight_col=0, 
         treeheight_row=0, color=hcl.colors(50, "Tropic"), clustering_method="ward.D2")
#dev.off()

write.csv(t(kk), "Fig_1I.csv")

In [ ]:
%%R

save_pheatmap_pdf <- function(x, filename, width=7, height=7) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}

xx <- pheatmap(t(kk),cluster_rows=F, cluster_cols=F, treeheight_col=0, 
         treeheight_row=0, color=hcl.colors(50, "Tropic"), clustering_method="ward.D2")

save_pheatmap_pdf(xx, "Figure_1I.pdf", width=10, height=2)